# Probabilistic Robotics: Assignment 5

A Particle Filter is a popular method used for simultaneous localization and mapping, because it is
relatively easy to implement, has computational advantages and is robust against data association
problems. Applying this to a scenario where only a limited number of landmarks are visible (i.e.
six colored landmarks around a soccer field), should be simple.

Yet, this time the algorithm has to be implemented to real robots. The observations of range
and bearing are not provided, but should be estimated from the camera images. This means that
the landmarks have to be detected, classified and localized in the image under changing lightingconditions.
This location in the image has to be converted to a vector in the local coordination
system by including the position of the head and the body of the robot. So, there will be a lot of
hard work, frustration and fun to get the particle filter working on the real robot.

The field of the RoboCup Soccer as used in the experiment. The Nao robot will walked
several times the 8-shaped path marked on the field:
<img src=https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/fieldSetup.jpg width="500">

For this experiment a configuration inspired the experiment by Steffen Gutmann is used [1]. Steffen
directed an Aibo over the field with a joy-stick, and recorded the odometry updates and the
observations of the landmarks of the Aibo. It is your task to do the same with a Nao robot. The
Nao has a limited field of view, so only 1-2 landmarks are visible at the same time. The clue to
detect landmarks is the pink band each landmark has. By looking above and below this band you
should be able to reliable classify the signature of the landmark.
Around the field six landmarks are visible, color coded with bands of pink combined with green,
yellow and blue (see Fig. ). The green landmarks are placed on locations (0,215) and (0,-215) cm.
The yellow landmarks are located at (315,215) and (315,-215) cm. The blue landmarks are placed
at (-315,215) and (-315,-215) cm. Note that you make the detection of the landmarks easier when you put white boards behind the landmarks.

These are the landmarks which are placed around the RoboCup Soccer field:
<img src=https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/Landmarks.jpg width="500">

On the field five black marks are made with black tape. Because the size of the soccer field is 6x4
meter, twice as large as the field used in the Gutmann dataset [1], the marks are placed further apart
(see Fig. ). The marks are placed in an eight-shape figure on locations (0,0), (0,100), (100,-100),
(-150,0) and (-150,100)cm.

The layout of the RoboCup Soccer field, with the markers on the field and the landmarks
around the field:
<img src=https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/FieldLayout.png width="500">

For your task a number of Nao humanoid robots with NaoQi are available: Tom, Jerry, Brooke, Mio, Eve, Walle, Sam and Moos. The other Nao robots in the lab have an hardware problem or are running the Dutch Nao Team
code. Note that Sam and Moos are v6 robots, while the other robots are v5 robots. The version is important for your choice of the software (NaoQi). An extensive introduction how to use the robot are available:

[extensive introduction](https://github.com/Caiit/DNT-Introduction/raw/master/DNT_Introduction.pdf)

Most important to remember the recommendation from Section 3.2 of the extensive introduction how to work with the robots:

>"Working with robots takes special care. Always work with at least two people
when working with the robot, since it is not that stable and can fall over a
lot. Make sure that one person is always standing next to the robot to catch it
when it falls and the other one behind the laptop to start and stop the code.
Furthermore, when not using the robot, let it go to its rest position by pressing
twice on its chest button. To undo this resting mode, again press twice on its
chest button. Also, always put the charger in the back of the robot when not
using it, its battery is not that great."

Remember that you can work in groups of 4 on this assignment.

# Exercise 1: Data Acquisition

Create a dataset by the following code or by combining the following Python scripts:
* [a script](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2017/NaoSLAM/keyboard_control.py) which drive the robot around with the keyboard (Courtesy  Michiel van der Meer and Caitlin Lagrand) 
* [a script](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2017/NaoSLAM/datasetCreator.py) to take pictures from the top camera (Courtesy Douwe van der Wal)
* you also need [the naomanager.py script](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2017/NaoSLAM//naomanager.py) (Courtesy Martin Stolk)
.

In [ ]:
import collections 
import time
import os
import datetime
import sys

from numpy.random import choice
import numpy as np

import math
import matplotlib
import matplotlib.pyplot as plt

import cv2
import matplotlib.image as mpimg
import urllib2
print (sys.version)

This should print the string '2.7.15rc1'. If it prints the string '3.6.6' you should do the following in the bash shell:

<tt>sudo python2 -m pip install ipykernel</tt>

followed by

<tt>python2 -m ipykernel install --user</tt>

Restart jupyter notebook and check if the python version is '2.7', which is required for NAOqi.

In [ ]:
class _Getch:
    """Gets a single character from standard input.  Does not echo to the
screen."""
    def __init__(self):
        try:
            self.impl = _GetchWindows()
        except ImportError:
            self.impl = _GetchUnix()

    def __call__(self): return self.impl()


class _GetchUnix:
    def __init__(self):
        import tty, sys

    def __call__(self):
        import sys, tty, termios
        fd = sys.stdin.fileno()
        old_settings = termios.tcgetattr(fd)
        try:
            tty.setraw(sys.stdin.fileno())
            ch = sys.stdin.read(1)
        finally:
            termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
        return ch


class _GetchWindows:
    def __init__(self):
        import msvcrt

    def __call__(self):
        import msvcrt
        return msvcrt.getch()


getch = _Getch()

++++++++++++++++++++++++++++Now you can download the SDKs to interface with the robot. You need both the Python and C++ SDK.
Go to

> [Softbank Robotics Community Resources](https://community.ald.softbankrobotics.com/en/resources/software/language/en-gb/robot/nao-2/field_software_type/sdk)

<img src=https://staff.fnwi.uva.nl/a.visser/research/nao/2018/NAOqiSDK.png width="500">

You could ask for the credentials from your TA. Alternatively, you could download the SDKs from:

> [Dutch Nao Team shared SDKs] 
(https://drive.google.com/open?id=1rGhWUx7I0UUsOKF0R72lVjt5LYPv-4Y3)

Download the Python 2.7 SDK 2.1.4 Linux 64 and the C++ SDK 2.1.4 Linux 64.

Install both SDKs in a directory <tt>packages</tt>. Remember the location of the directory, you need that location for jupyter and your bash shell.

Edit your <tt>~/.bash_profile</tt>. Add a few lines. First line to add is:
```
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/x86_64-linux-gnu/</tt>
```


Followed by
```
# NaoQi 2.1.4.13 for Nao v5
export PYTHONPATH=$PYTHONPATH:$HOME/packages/aldabaran/pynaoqi-python2.7-2.1.4.13-linux64
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/packages/aldebaran/naoqi-sdk-2.1.4.13-linux64/lib:$HOME/packages/aldabaran/pynaoqi-python2.7-2.1.4.13-linux64
export JUPYTER_PATH=/$HOME/packages/aldabaran/pynaoqi-python2.7-2.1.4.13-linux64
```

Load this profile with the command <tt>source ~/.bashrc</tt>. Check it with <tt>env | grep PATH</tt>

Restart the Jupyter notebook and check if the following code loads.

In [ ]:
sys.path.append("/home/qwerthjkl45/packages/pynaoqi-python2.7-2.4.3.28-linux64")
from naoqi import ALProxy


In [ ]:
DEFAULT_PORT = 9559

class Nao(object):
    """ Represents a Nao. Initializes the proxies we want.
        Additionally provides methods for logging the angles."""
    def __init__(self, ip, port=9559, logfolder=None):
        super(Nao, self).__init__()

        self.ip = ip
        self.port = port

        self.motion = ALProxy('ALMotion', ip, port)
        self.behavior = ALProxy('ALBehaviorManager', ip, port)
        self.posture = ALProxy('ALRobotPosture', ip ,port)

        self.jointnames = self.motion.getJointNames("Body")
        self.timestamps = []
        self.desiredangles = defaultdict( lambda : list() )
        self.realangles = defaultdict( lambda : list() )

        self.initstiffness()

        # Create logging folder if needed
        self.logfolder = logfolder
        self.logfp = None
        if logfolder:
            if not os.path.exists(logfolder):
                os.mkdir(logfolder)

    def initstiffness(self):
        self.motion.stiffnessInterpolation('Body', 0.5, 1.0)

    def releasestiffness(self):
        self.motion.stiffnessInterpolation('Body', 0.0, 1.0)

    def startlogging(self):
        if not self.logfolder:
            return
        now = datetime.datetime.now()
        self.logname = '%s_%s-%.4d%.2d%.2d%.2d%.2d%.2d' % (self.ip, self.port, now.year, now.month, now.day, now.hour, now.minute, now.second)
        self.logfp = open(os.path.join(self.logfolder, self.logname), 'wb')

        # First line of the log file contains the joint names
        self.logfp.write('timestamp')
        for n in self.jointnames:
            self.logfp.write('\t%s\t ' % (n))
        self.logfp.write('\n')

    def stoplogging(self):
        if self.logfp:
            self.logfp.close()
            self.logfp = None

    def stop(self):
        self.releasestiffness()
        self.motion = None
        self.behavior = None
        self.posture = None
        self.stoplogging()

    def updateanglehistory(self):
        self.timestamps.append(time.time())
        if self.logfolder:
            self.logfp.write('%f' % (self.timestamps[-1]))

        n = len(self.timestamps)
        self.timestamps = self.timestamps[max(0,n-100):n]

        angles = self.motion.getAngles('Body', False)
        anglesreal = self.motion.getAngles('Body', True)
        for i, name in enumerate(self.jointnames):
            self.desiredangles[name].append(angles[i])
            self.realangles[name].append(anglesreal[i])

            n = len(self.desiredangles[name])
            self.desiredangles[name] = self.desiredangles[name][max(0,n-100):n]
            n = len(self.realangles[name])
            self.realangles[name] = self.realangles[name][max(0,n-100):n]

            if self.logfolder:
                self.logfp.write('\t%f\t%f' % (angles[i], anglesreal[i]))
        if self.logfolder: self.logfp.write('\n')
            

    def gettimestamps(self): return self.timestamps

class NaoCallAll(object):
    """ Wraps a call to multiple Naos/attributes.
        Returns a new call wrapper in case you try to access an attribute."""
    def __init__(self, objects, attr):
        super(NaoCallAll, self).__init__()

        self.attrobjects = []

        for o in objects:
            self.attrobjects.append( o.__getattribute__(attr) )

    def __getattr__(self, attr):

        return NaoCallAll(self.attrobjects, attr)

    def __call__(self, *args, **kwargs):
        results = []
        for a in self.attrobjects:
            results.append( a(*args, **kwargs) )
        return results

    def __getitem__(self, k):
        results = []
        for a in self.attrobjects:
            results.append( a.__getitem__(k) )
        return results


class NaoManager(list):
    """ Manages a list of nao's.
        Works like a normal list, except a call on the
        list will result in the method being called on all nao's in the list."""
    def __init__(self, *args, **kwargs):
        super(NaoManager, self).__init__(*args, **kwargs)

    def __getattr__(self, attr):
        return NaoCallAll(self, attr)

    def addnao(self, ip, port=9559, logfolder=None):
        self.append( Nao(ip, port, logfolder) )

In [ ]:
"""
keyboard_control.py:
    Control a Nao robot that is currently running the Naoqi software with your
    keyboard! Key bindings are defined in controllers.txt.

    Requires naomanager.py & an installation of pynaoqi
"""

import argparse
import sys
import time
import math
#import getch
import json
from threading import Thread

#from naomanager import NaoManager, DEFAULT_PORT
from naoqi import ALProxy

__author__ = "Michiel van der Meer, Caitlin Lagrand"
__copyright__ = "Copyright 2017, Dutch Nao Team"
__version__ = "0.2.0"

#getch = getch._Getch()

naos = NaoManager()

def load_motion(name):
    print("Performing motion {}".format(name))
    with open("animations/{}.json".format(name), "r") as f:
        fdict = json.load(f)
        fdict["names"] = [str(x) for x in fdict["names"]]
    return fdict


# Main program to perform actions
def main(args):
    # Create list of nao objects
    
    if hasattr(args, 'nao'):
        for nao in args.nao:
            try:
                ip, port = nao.split(':')
                # naos.addnao(ip, int(port))
            except ValueError:
                ip = nao
                port = DEFAULT_PORT
                naos.addnao(ip, int(port))
    else:
        naos.add("nao.local", DEFAULT_PORT)
    
    if len(naos) == 1:
        print("Connected {} Nao".format(len(naos)))
    else:
        print("Connected {} Naos".format(len(naos)))
    x = 0.0
    y = 0.0
    theta = 0.0
    frequency = 0.3
    CommandFreq = 0.5
    starttime = time.time()
    print("Ready for liftoff at: {}".format(starttime))

   


In [ ]:
main("moos.local")

for nao in naos:
    nao.startlogging()

In [ ]:
IP = "moos.local"
PORT = 9559

def start_proxies():
    '''
    Start the motion and posture proxy and let the robot stand up.
    '''
    motion_proxy  = ALProxy("ALMotion", IP, PORT)
    posture_proxy = ALProxy("ALRobotPosture", IP, PORT)
    memory_proxy  = ALProxy("ALMemory", IP, PORT)

    motion_proxy.wakeUp()
    posture_proxy.goToPosture("StandInit", 0.5)
    motion_proxy.setAngles("HeadPitch", 0.3, 0.1)

    return motion_proxy, posture_proxy, memory_proxy

def create_video_connection(ip=None, port=None, camera=0):
    '''
    Create a connection with the robot and start a camera proxy.
    returns the video device and the capture device.
    https://gist.github.com/takamin/990aa0133919aa58944d
    '''
    if ip == None: ip = IP
    if port == None: port = PORT
    # Create proxy to nao
    cam_proxy = ALProxy("ALVideoDevice", ip, port)
    AL_kTopCamera = camera # 0: topcamera, 1: bottomcamera
    AL_kQVGA = 1      # 320x240
    AL_kBGRColorSpace = 13
    return cam_proxy, cam_proxy.subscribeCamera("top", AL_kTopCamera, AL_kQVGA, AL_kBGRColorSpace, 10)


def get_img_from_robot(video_device, capture_device):
    '''
    Get a frame from the robot and return it as numpy array.
    https://gist.github.com/takamin/990aa0133919aa58944d
    '''
    # Create image
    width = 320
    height = 240
    img = np.zeros((height, width, 3), np.uint8)
    captured = video_device.getImageRemote(capture_device)
    if captured == None:
        print("Cannot capture")
    elif captured[6] == None:
        print("No image data string")
    else:
        # translate value to mat
        values = map(ord, list(captured[6]))
        i = 0
        for y in range(0, height):
            for x in range(0, width):
                img.itemset((y, x, 0), values[i + 0])
                img.itemset((y, x, 1), values[i + 1])
                img.itemset((y, x, 2), values[i + 2])
                i += 3
    return img


motion_proxy, posture_proxy, memory_proxy = start_proxies()

In [ ]:


width = 320
height = 240
image = np.zeros((height, width, 3), np.uint8)
num = 200

video_device, capture_device = create_video_connection(IP, PORT, 0)

In [ ]:
a = memory_proxy.getDataList("Device/Laser/Value")


If ```import cv2``` fails, quit jupyter and execute the command:

```
sudo pip install opencv-contrib-python
```

In [ ]:
# capture an image
# import almath

for nao in naos:
    nao.updateanglehistory()
   
jointvalues = motion_proxy.getAngles("Body", True)
jointnames = motion_proxy.getBodyNames("Body")
position = motion_proxy.getRobotPosition(False)
    
result = video_device.getImageRemote(capture_device);
if result == None:
    print 'cannot capture.'
elif result[6] == None:
    print 'no image data string.'
else: 
    print 'captured.'
    values = map(ord, list(result[6]))
    image = np.reshape(values, (height, width, 3)).astype('uint8')
    cv2.imshow("top-camera-320x240", image)
    string2 = "./plaatje" + str(num) + ".jpg"
    cv2.imwrite(string2, image) 
    
    string2 = "./plaatje" + str(num) + ".txt"
    file = open(string2,"w")
    
    file.write("Recording at estimated position: " + str(position) + "\n")
    
    for jointvalue, jointname in zip(jointvalues, jointnames):
        file.write(jointname + " " + str(jointvalue) + "\n")
    file.close()
    num += 1
    

In [ ]:
def record_data(capture_device, motion_proxy, naos, num, mark = False):
    file = open('datalog.txt', "a")
    
    if not mark:
        position = motion_proxy.getRobotPosition(False)
        lm, r, b = capture_an_image(capture_device, motion_proxy, naos, num)
        #lm, r, b = detect_landmark('./plaatje100.jpg')
        #position = [1, 2, 3] # list
        file.write("obs: " + str(position[0]) + " "  + str(position[1]) + " " + str(position[2]))
        if lm == "":
            file.write(" "+ str(0)+ "\n")
        else:
            file.write(" "+ str(1) + " ( " + lm + " " + str(r) + " " + str(b) + " )"+ "\n")
    else:
        file.write('mark\n')
    file.close()
#record_data()

In [ ]:
def capture_an_image(capture_device, motion_proxy, naos, num):
    lm = ""
    for nao in naos:
        nao.updateanglehistory()
   
    jointvalues = motion_proxy.getAngles("Body", True)
    jointnames = motion_proxy.getBodyNames("Body")
    position = motion_proxy.getRobotPosition(False)

    result = video_device.getImageRemote(capture_device);
    if result == None:
        print 'cannot capture.'
    elif result[6] == None:
        print 'no image data string.'
    else: 
        print 'captured.'
        values = map(ord, list(result[6]))
        image = np.reshape(values, (height, width, 3)).astype('uint8')
        cv2.imshow("top-camera-320x240", image)
        string2 = "./plaatje" + str(num) + ".jpg"
        cv2.imwrite(string2, image)
        lm, r, b = detect_landmark(string2)
        
    return lm,r, b
        


In [ ]:
# need to change capture_device blablabla
#capture_device, motion_proxy, naos, 
num = 0
while True:
    key_press = raw_input()
    if (key_press == 'w'):
        print("Moving forward")
        x = 0.5; CommandFreq = 0.5; frequency = 0.3;
        motion_proxy.setWalkTargetVelocity(x, 0, 0, frequency)
        time.sleep(CommandFreq)
        motion_proxy.setWalkTargetVelocity(0, 0, 0, frequency)
        time.sleep(CommandFreq)
    elif (key_press == 's'):
        print("Standup")
        posture_proxy.goToPosture("StandInit", 0.5)
    elif (key_press == 'x'):
        print("Stopping")
        x = 0.5; CommandFreq = 0.5; frequency = 0.3;
        motion_proxy.setWalkTargetVelocity(0, 0, 0, frequency)
        time.sleep(CommandFreq)
    elif (key_press == 'a'):
        print("Turning left")
        x = 0.0; y = 0.0; theta = 0.5; CommandFreq = 0.5; frequency = 0.3;
        motion_proxy.setWalkTargetVelocity(x, y, theta, frequency)
        time.sleep(CommandFreq)
        motion_proxy.setWalkTargetVelocity(0, 0, 0, frequency)
        time.sleep(CommandFreq)
    elif (key_press == 'd'):
        print("Turning right")
        x = 0.0; y = 0.0; theta = -0.5; CommandFreq = 0.5; frequency = 0.3;
        motion_proxy.setWalkTargetVelocity(x, y, theta, frequency)
        time.sleep(CommandFreq)
        motion_proxy.setWalkTargetVelocity(0, 0, 0, frequency)
        time.sleep(CommandFreq)
    elif (key_press == 'r'):
        print('record data')
        record_data(capture_device, motion_proxy, naos, num)
        num = num + 1
    elif (key_press == 'm'):
        print('record data mark')
        record_data(capture_device, motion_proxy, naos, num, True)
        num = num + 1
    elif (key_press == 'z'):
        print("Going to rest")
        motion_proxy.post.rest()
        time.sleep(0.5)
        print("Resting (zzz)")
        motion_proxy.stiffnessInterpolation('Body', 0.0, 1.0)
    elif (key_press == 'c'):
        print("Closing Connection")
        CommandFreq = 0.5;
        
        record_data(capture_device, motion_proxy, naos, num)
        for nao in naos:
            nao.stoplogging()
            nao.motion.rest()
            time.sleep(CommandFreq)
            nao.motion.killAll()
            nao.stop()
        break
        
    elif (key_press == 'p'):
        break
    else:
        print('doing nothing')

# Exercise 2: Detect the landmarks

Find the landmarks by searching for colored regions. Check the size, shape and location above the horizon
to be sure that you have a landmark. Check the color above and below the pink band to get the signature of the observation $s^i_t$. Estimate the range $r^i_t$ from the size of the pink object. Estimate the bearing $\phi^i_t$ by combining the center of the object in the image with the HeadPitch angle of the Nao
robot. Try to get statistics on the standard deviation of your observations.
* you could use [the ColorRegion.py script](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2017/NaoSLAM//ColorRegion.py) (Courtesy Sebastian Thrun) as starting point.


In [ ]:
# 0 -> green 
#1 -> magenta 
#2 -> yellow 
#3 -> blue
def find_color(color_range, crop_img):
    landmark = ""
    x, y, _ = crop_img.shape
    area_list = np.zeros((3, ))
    y_list = np.zeros((3, ))
    for idx in range(3):
        lower = np.uint8(color_range[(idx*2) + 1, :])
        upper = np.uint8(color_range[idx*2 , :])
        mask2 = cv2.inRange(crop_img, lower, upper)
        #mask2 = cv2.inRange(crop_img, gl, gh)
        #mask2 = cv2.erode(mask2, kernel)
        
        kernel1 = np.ones((8, 8), "uint8")
        mask2 = cv2.dilate(mask2, kernel1)
        _, contours, _ = cv2.findContours(mask2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                
        if len(contours) == 0:
            continue
        
        cnt2 = max(contours, key = cv2.contourArea)
        if (cv2.contourArea(cnt2) > 100):
            x1, y1, _, _ = cv2.boundingRect(cnt2)
            
            area_list[idx] = cv2.contourArea(cnt2)
            y_list[idx] = y1
            
   
    if not (area_list == 0).all():
        idx = np.argmax(area_list)
        y1 = y_list[idx]
        if idx == 0:
            if y1 < y/2:
                print('yp')
                landmark = "2:1"
            else:
                print('py')
                landmark = "1:2"
        elif idx == 1:
            if y1 < y/2:
                print('bp')
                landmark = "0:1"
            else:
                print('pb')
                landmark = "1:0"
        else:
            if y1 < y/2:
                print('gp')
                landmark = "3:1"
            else:
                print('pg')
                landmark = "1:3"
    else:
        print('detect nothing')

    
    return landmark

In [ ]:
def calculate_range_bearing(x,w, pic_width):
    z_range = 30.0 * 1290.5/w #(mm)
    
    # calculate focal  length (pixel)
    foc = np.tan((90 - (60.97/2)) * np.pi/(180)) * pic_width /2
    try:
        z_bearing = 90 - (np.arctan(float(foc) / abs(x + (w/2) - (pic_width/2)))* 180/np.pi)
    except ZeroDivisionError:
        z_bearing = 90 - (np.arctan(np.inf)* 180/np.pi)
    return z_range, z_bearing
    

In [ ]:
def detect_landmark(file_str):
    lm = ''
    r = -10000
    b = -10000
    
    wh = np.array([50, 50, 50], np.uint8)
    wl = np.array([0, 0, 0], np.uint8)


    mgh = np.array([170 ,255 ,255], np.uint8)
    mgl = np.array([140 ,50  ,0], np.uint8)

    color_range = np.zeros((6, 3), dtype= np.uint8)

    yh = np.array([40 ,255 ,255], np.uint8) #yh
    yl = np.array([15 ,30 ,30], np.uint8) #yl

    color_range[0, :] =  np.array([40, 255, 255], np.uint8) #yh
    color_range[1, :] = np.array([15, 25, 25], np.uint8) #yl

    color_range[2, :] = np.array([150,255,255], np.uint8) #bh
    color_range[3, :] = np.array([100,100,100], np.uint8) #bl


    color_range[4, :] = np.array([80 ,200 ,200], np.uint8) #gh 80 ,100 ,100
    color_range[5, :] = np.array([45 ,50 ,0], np.uint8) # gl




    #plaatje205.jph
    img = cv2.imread(file_str)
    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape
    #image = mpimg.imread(f,format='jpg')

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Threshold the HSV image to get only blue colors
    #mask = cv2.inRange(hsv, yl, yh)
    mask1 = cv2.inRange(hsv, mgl, mgh)

    #mask3 = cv2.inRange(img1, wl, wh)
    kernel = np.ones((4, 4), "uint8")
    kernel1 = np.ones((8, 8), "uint8")
    mask1 = cv2.erode(mask1, kernel)
    mask1 = cv2.dilate(mask1, kernel1)

    #mask = cv2.bitwise_or(mask1, mask2)
    #target = cv2.bitwise_and(img1,img1, mask=mask)

    _, contours, _ = cv2.findContours(mask1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv2.contourArea)

    x, y, w, h = cv2.boundingRect(cnt)
    

    if (y - h)> 0 and ((y + 2*h) < height):
        # first check 
        crop_img = hsv[y - h :y+(2*h), x:x+w]
        lm = find_color(color_range, crop_img)

    elif ((y - h)< 0) and ((y + 2*h) < height):
        crop_img = hsv[y :y+(2*h), x:x+w]
        #crop_img = hsv[y :y+(2*h), x:x+2*w]
        lm = find_color(color_range, crop_img)
    else: 
        crop_img = hsv[y - h:y, x:x+(1*w)]
        lm = find_color(color_range, crop_img)

    if not(lm == ""):
        r, b = calculate_range_bearing(x, w, width)
    return lm, r, b

A number of example images are available for testing (Courtesy Tobias Garritsen):
* [yp.jpg](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/yp.jpg)
* [py.jpg](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/py.jpg)
* [gp.jpg](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/gp.jpg)
* [pg.jpg](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/pg.jpg)
* [bp.jpg](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/bp.jpg)
* [pb.jpg](https://staff.fnwi.uva.nl/a.visser/education/ProbabilisticRobotics/2018/pb.jpg)

In [ ]:
detect_landmark('./plaatje201.jpg')
        

In [ ]:
-7.7*180/np.pi

In [ ]:
np.tanh(94/)

# Exercise 3:  FastSLAM

Implement the FastSLAM 1.0 algorithm of Table 13.1 of the book. Analyze the performance and suggest possible
improvements.

In [ ]:
# need to aware that theta is radius in log not degree 

In [ ]:
class Paticle:
    def __init__(self, num_particles, num_landmark):
        self.x = np.zeros((3, )) # x, y, theta
        self.landmarks_mu = np.zeros((2* num_landmark)) # landmarks_x, landmarks_y
        self.landmarks_Sigma = np.zeros((num_landmark, 2, 2))
        #X_0= np.random.multivariate_normal([0, 0, 0], [[0.5, 0, 0],[0, 0.5, 0], [0, 0, 0.5]], 1)
        self.x[0:3] = np.array([0, 0, 0])
        self.weight = 1/float(num_particles)
        self.seen_before = [False for t in range(num_landmark)]
        
    def __getitem__(self,key):
        print(key)
        return getattr(self,key)


In [ ]:

def parse_line(s):        
    s = s.split(' ')
    if s[0] == 'mark':
        return None, None

    if s[0] == 'obs:':
        x_obs = [float(s[2]) / 100,  # milimeters to decimeters
                 float(s[3]) / 100,
                 float(s[4]) * np.pi / 180] # degrees to radians

        n_obs = int(s[5])
        z_obs = np.zeros((2, n_landmarks))
        for i in range(n_obs):
            idx = 6 + i * 5
            signature, x, y = s[idx+1:idx+4]                
            signature = int(signature[0]), int(signature[2])
            x = float(x) / 100 # milimeters to decimeters
            y = float(y) * np.pi / 180 # degrees to radians

            for landmark in range(n_landmarks):
                if signature[0] == LID[0, landmark] and signature[1] == LID[1, landmark]:
                    z_obs[:, landmark] = [x, y]

        return x_obs, z_obs 
    print(s[0])
    return None, None

def distance(ax, bx, ay, by):
    return np.sqrt((ax - bx)**2 + (ay - by)**2)

def state_transition(x_, ut):
    
    # x_: 3x1 
    v = ut[0]  # translation
    da = ut[1] # delta angle
    sa = ut[2] # start angle
    
    new_ut= np.random.multivariate_normal([v, da, sa], [[0.005, 0, 0],[0, 0.01, 0], [0, 0, 0.005]], 1)
    
    v = new_ut[:, 0]  # translation
    da = new_ut[:, 1] # delta angle
    sa = new_ut[:, 2] # start angle
    
    # predicted robot position mean
    x_ = np.array(
            [x_[0] + v*np.cos(x_[2]+sa),
             x_[1] + v*np.sin(x_[2]+sa),
             x_[2]+da]).reshape((3,))
    
    return x_

def calculate_Jacobian(particle, landmark_id):
    x = landmark_id * factor_dim
    y = landmark_id * factor_dim + 1
    
    delta_x = particle.landmarks_mu[x] - particle.x[0]
    delta_y = particle.landmarks_mu[y] - particle.x[1]
    
    expect_range =  np.sqrt((delta_x)**2 + (delta_y)**2)
    expect_bearing = normalized_angle(np.arctan2(delta_y, delta_x) - particle.x[2])
    
    h = np.array([expect_range, expect_bearing ])
    H = np.array(
        [[delta_x/expect_range, delta_y/expect_range],
         [-delta_y/(expect_range**2),  delta_x/(expect_range**2)]]).reshape(2, 2)
    return h, H

def normalized_angle(z_bearing):
    if (z_bearing >np.pi) or (z_bearing<-np.pi):
        while (z_bearing > np.pi):
            z_bearing = z_bearing - (2*np.pi);

        while (z_bearing < -np.pi):
            z_bearing = z_bearing + (2*np.pi);
    
    return z_bearing
    

def measurement_update_SLAM(prev_particle, particle, zt):
    H = np.zeros((factor_dim, dim, n_landmarks))    
    foundx = np.zeros((dim, n_landmarks ))
    K = np.zeros((dim, factor_dim, n_landmarks))
    for landmark in range(n_landmarks):
        
        x_idx =landmark * factor_dim
        y_idx =landmark * factor_dim + 1            
        
        particle.landmarks_mu[x_idx] = prev_particle.landmarks_mu[x_idx]
        particle.landmarks_mu[y_idx] = prev_particle.landmarks_mu[y_idx]
        particle.landmarks_Sigma[landmark, :] = prev_particle.landmarks_Sigma[landmark, :] 
        
              
        if zt[0, landmark] != 0:   # if Landmark is observed
            
            
            
            x_idx =landmark * factor_dim
            y_idx =landmark * factor_dim + 1
            if particle.landmarks_mu[x_idx] == 0:   # if landmark is never seen before:                
                particle.landmarks_mu[x_idx] = particle.x[0] + (zt[0, landmark]*np.cos(zt[1, landmark] + particle.x[2]))
                particle.landmarks_mu[y_idx] = particle.x[1] + (zt[0, landmark]*np.sin(zt[1, landmark] + particle.x[2]))   
                
                # initialize mean
                h, H = calculate_Jacobian(particle, landmark)
                inv_H = np.linalg.inv(H)
                particle.landmarks_Sigma[landmark, :] = (inv_H.dot(Q)).dot(inv_H.T)
                
                # weight still stay the same as defult importance weight
                
            else:
               
                
                ecptected_z, H = calculate_Jacobian(particle, landmark)
                sig = particle.landmarks_Sigma[landmark, :]
                Q_ = ((H.dot(sig)).dot(H.T)) + Q
                inv_Q_ = np.linalg.inv(Q_)
                # calculate the gain K
                K = (sig.dot(H.T)).dot(inv_Q_)
                z_diff = zt[:, landmark] - ecptected_z
                z_diff[1] = normalized_angle(z_diff[1])
                # update mean
                particle.landmarks_mu[x_idx: y_idx + 1] = particle.landmarks_mu[x_idx: y_idx + 1] + K.dot(z_diff)
                particle.landmarks_Sigma[landmark, :] = particle.landmarks_Sigma[landmark, :] - (K.dot(H)).dot(sig)
                
                # calculate the weigh
                particle.weight = particle.weight* np.exp(-1/2* (z_diff.T.dot(np.linalg.inv(Q_)).dot(z_diff)))\
                                  /np.sqrt(np.linalg.det(2 * np.pi* Q_))
                
            
    return particle

In [ ]:
logfilename = 'data/dlog.dat'; N = 51666;
dim = 3
factor_dim = 2
#------------------------------------------------------------ data creation
#
# true robot position at t = 1
xt = np.zeros((dim, N)) # x[:, i] = [x y angle]

# user input at t = 1
u = np.zeros((3, N))              # u = [trans, delta_angle, start_angle]

# Landmark locations
L2006 = np.array([[20,  20, -20, -20],
                  [20, -20,  20, -20]])

# You also need the following information about the landmark positions: 
# cyan:magenta -1500 -1000 magenta:cyan -1500 1000 magenta:green 0 -1000 green:magenta 0 1000 yellow:magenta 1500 -1000 magenta:yellow 1500 1000 
# 0 -> green 1 -> magenta 2 -> yellow 3 -> blue
L = np.array([[-15, -15,   0,  0,  15, 15],
              [-10,  10, -10, 10, -10, 10]])
LID = np.array([[3, 1, 1, 0, 2, 1], 
                [1, 3, 0, 1, 1, 2]])

# expected robot location noise
m_err = .1;
Q = m_err*np.eye(2); 

z = np.zeros((2, N, L.shape[1]))
n_landmarks = 6

mark_t = []

with open(logfilename, 'r') as f:
    lines = f.read().split('\n')
    for t, line in enumerate(lines):
        x_obs, z_obs = parse_line(line)

        if x_obs is None:
            if t > 1 and t < N:
                mark_t += [t]
                xt[:,t] = xt[:,t-1] # skip the observation with a mark, use previous measurement
                z[:,t] = z[:,t-1]
            continue
            
        xt[:, t] = x_obs
        z[:, t] = z_obs

        if t > 0:
            dx=xt[0,t]-xt[0,t-1]
            dy=xt[1,t]-xt[1,t-1]

            u[2,t] = np.arctan2(dy,dx) - xt[2,t-1] # start_angle
            u[2,t] = np.mod(u[2,t] + np.pi, 2*np.pi) - np.pi
            u[1,t] = xt[2,t]-xt[2,t-1]; # diff_angle
            u[1,t] = np.mod(u[1,t] + np.pi, 2*np.pi) - np.pi # FIX angle difference range
            u[0,t] = np.sqrt(dx*dx+dy*dy) # translation


#---------------------------------------------------------------- a prioris
x_ = xt[:, 1] # a priori x = true robot position


In [ ]:
def draw_samples_from_newset(particles, sampling_number):
    elements = np.arange(sampling_number);
    #weights = X[:, 2]/X[:, 2].sum();
    weights = np.zeros((sampling_number, ))
    for idx in range(sampling_number):
        weights[idx] = (particles[idx].weight)
     
    weights = weights/weights.sum()
    for idx in range(sampling_number):
        particles[idx].weight = weights[idx]
        
    idx_chosen = (choice(elements, p=weights, size=sampling_number));
    
    new_particles = []
    for idx, idx_val in enumerate(idx_chosen):
        particles[idx_val].weight = 1.0/M
        new_particles += [particles[idx_val]]
    return new_particles



In [ ]:
M = 100; #sampling number
T = 5000  # total timestamp
Features = 6

mark_ts = mark_t[-6:-1]
mark_p = np.zeros((5, 3))
mark_ts_idx = 0

particles = [[Paticle(num_particles=M, num_landmark=Features) for x in range(M)] for y in range(2)]
for t in range(N-1):
    #print('posterior at time = ', t+1, ' before updating with measurement');
    
    for m in range(M):  # loop over all particles
        # calculate transition probaility:
        # update state
        
        particles[1][m].x = state_transition(particles[0][m].x, u[:, t])
        particles[1][m].x[2] = normalized_angle(particles[1][m].x[2])
        particles[1][m] = measurement_update_SLAM(particles[0][m], particles[1][m], z[:, t, :])
        
    # initilize new particle set    
    particles[0] = draw_samples_from_newset(particles[1], M)
    particles[1] = [Paticle(num_particles=M, num_landmark=Features) for x in range(M)]
    
    if mark_ts[mark_ts_idx] == t:
        mark_p[mark_ts_idx % 5, :] = particles[0][0].x
        print(mark_p)
        mark_ts_idx = mark_ts_idx + 1
    
    if (t % 1000) == 0:
        print(t)
    if ((mark_ts_idx% 5) == 0 and not (mark_ts_idx == 0)) or ((t % 5000) == 0):
        fig, ax = plt.subplots()
        for idx in range(80):
            ax.plot(particles[0][idx].landmarks_mu[0:12:2],particles[0][idx].landmarks_mu[1:12:2], 'bo')#, label="Prediction");   
            if mark_ts_idx == 5:
                ax.plot(mark_p[:, 0],mark_p[:, 1], 'ro')#, label="Prediction");       
        ax.set_xlim(-20, 20)
        ax.set_ylim(-20, 20)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("Visualization of uncertainty of localization of EKL for SLAM")
        plt.legend()
        plt.show()
        if mark_ts_idx == 5:
            break
    

#  Hand-In

You can work on this assignment up to four people. When you have completed the
assignment, each should upload your solution to Canvas (with clear pointers to your team-mates).